<a href="https://colab.research.google.com/github/Neuro1729/Mechanistic-Interpretability/blob/main/MI1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Model name on Hugging Face Hub
model_name = "microsoft/phi-2"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,   # use float16 to save memory
    device_map="auto"            # automatically put layers on available GPU
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
prompt = "Describe how the concept of refraction is used in the design of eyeglasses."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Describe how the concept of refraction is used in the design of eyeglasses.

Solution:
The concept of refraction is used in the design of eyeglasses to correct vision problems. Eyeglasses consist of lenses that refract light in a way that compensates for the specific refractive errors of the wearer's eyes.

For individuals with nearsightedness (myopia), the lenses in eyeglasses are concave. This shape causes light rays to diverge before they enter the eye, allowing them to focus properly on the retina. By refracting light in this way, eyeglasses help to correct the blurred vision associated with myopia.

On the other hand, individuals with farsightedness (hyperopia) have difficulty focusing on nearby objects. The lenses in eyeglasses for farsightedness are convex. This shape causes light rays to converge before they enter the eye, allowing them to focus properly on the retina. By refracting light in this way, eyeglasses help to correct the blurred


In [8]:
import random


# ---- Step 1: Define a forward hook to zero out random neurons ----
ablated_neurons = {}  # store which neurons are ablated per layer

def ablate_neurons(layer_name, x):
    # x: [batch, seq_len, hidden_dim]
    batch, seq_len, hidden_dim = x.shape
    n_ablated = int(0.1 * hidden_dim)  # 10%

    # Pick neurons randomly for this layer if not already picked
    if layer_name not in ablated_neurons:
        ablated_neurons[layer_name] = random.sample(range(hidden_dim), n_ablated)

    x_clone = x.clone()
    x_clone[:, :, ablated_neurons[layer_name]] = 0
    return x_clone

# ---- Step 2: Register hooks for MLP outputs in all layers ----
hooks = []
for i, layer in enumerate(model.model.layers):
    def get_hook(idx):
        return lambda module, input, output: ablate_neurons(f"layer{idx}", output)

    h = layer.mlp.fc2.register_forward_hook(get_hook(i))
    hooks.append(h)

# ---- Step 3: Run generation ----
prompt = "Describe how the concept of refraction is used in the design of eyeglasses."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# ---- Step 4: Remove hooks ----
for h in hooks:
    h.remove()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Describe how the concept of refraction is used in the design of eyeglasses.

Solution:
The concept of refraction is used in the design of eyeglasses to correct vision problems. Eyeglasses have lenses that are specifically shaped to bend light in a way that compensates for the refractive errors in the eye.

For example, if a person has nearsightedness (myopia), the light entering the eye focuses in front of the retina instead of directly on it. This causes distant objects to appear blurry. In eyeglasses, the lenses are concave, meaning they are thinner at the center and thicker at the edges. This shape causes the light rays to diverge before entering the eye, allowing them to focus properly on the retina.

Similarly, if a person has farsightedness (hyperopia), the light entering the eye focuses behind the retina instead of directly on it. In eyeglasses, the lenses are convex, meaning they are thicker at the center and thinner at the edges. This shape causes the light


In [9]:
import random


# ---- Step 1: Define a forward hook to zero out random neurons ----
ablated_neurons = {}  # store which neurons are ablated per layer

def ablate_neurons(layer_name, x):
    # x: [batch, seq_len, hidden_dim]
    batch, seq_len, hidden_dim = x.shape
    n_ablated = int(0.5 * hidden_dim)  # 10%

    # Pick neurons randomly for this layer if not already picked
    if layer_name not in ablated_neurons:
        ablated_neurons[layer_name] = random.sample(range(hidden_dim), n_ablated)

    x_clone = x.clone()
    x_clone[:, :, ablated_neurons[layer_name]] = 0
    return x_clone

# ---- Step 2: Register hooks for MLP outputs in all layers ----
hooks = []
for i, layer in enumerate(model.model.layers):
    def get_hook(idx):
        return lambda module, input, output: ablate_neurons(f"layer{idx}", output)

    h = layer.mlp.fc2.register_forward_hook(get_hook(i))
    hooks.append(h)

# ---- Step 3: Run generation ----
prompt = "Describe how the concept of refraction is used in the design of eyeglasses."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# ---- Step 4: Remove hooks ----
for h in hooks:
    h.remove()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Describe how the concept of refraction is used in the design of eyeglasses.
Solution:
The concept of refraction is used in the design of eyeglasses to correct vision problems. When light enters the eye, it is refracted by the lens before it reaches the retina. In a normal eye, the lens refracts the light evenly, allowing the light to reach the retina in a straight path. However, in a person with a vision problem, the lens may refract the light unevenly, causing the light to reach the retina in a curved path. This can result in a blurry or distorted image. By using lenses with different refractive properties, such as lenses with a curved shape or lenses with a flat shape, eyeglasses can redirect the light to refract in a different way, allowing the light to reach the retina in a straight path and correct the vision problem.

Follow-up Exercise 3:

Explain how the concept of refraction is used in the design of a telescope.
Solution:
The concept of ref


In [11]:
import random


# ---- Step 1: Define a forward hook to zero out random neurons ----
ablated_neurons = {}  # store which neurons are ablated per layer

def ablate_neurons(layer_name, x):
    # x: [batch, seq_len, hidden_dim]
    batch, seq_len, hidden_dim = x.shape
    n_ablated = int(0.7 * hidden_dim)  # 10%

    # Pick neurons randomly for this layer if not already picked
    if layer_name not in ablated_neurons:
        ablated_neurons[layer_name] = random.sample(range(hidden_dim), n_ablated)

    x_clone = x.clone()
    x_clone[:, :, ablated_neurons[layer_name]] = 0
    return x_clone

# ---- Step 2: Register hooks for MLP outputs in all layers ----
hooks = []
for i, layer in enumerate(model.model.layers):
    def get_hook(idx):
        return lambda module, input, output: ablate_neurons(f"layer{idx}", output)

    h = layer.mlp.fc2.register_forward_hook(get_hook(i))
    hooks.append(h)

# ---- Step 3: Run generation ----
prompt = "Describe how the concept of refraction is used in the design of eyeglasses."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# ---- Step 4: Remove hooks ----
for h in hooks:
    h.remove()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Describe how the concept of refraction is used in the design of eyeglasses.


Solution:

The answer of the exercise exercise is " " " " " " " " " " ".



In [12]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): LayerNorm((2560,), eps=1

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn.functional as F

# --- Load model and tokenizer ---
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [11]:
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()

prompt = "Describe how the concept of refraction is used in the design of eyeglasses."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generation settings
max_new_tokens = 50
temperature = 0.7
top_p = 0.9
results = {}

# --- 2. Store Original Layers and Set Up Experiment ---
# Access the ModuleList containing the 32 decoder layers
original_layers = model.model.layers
num_total_layers = len(original_layers)

print(f"Starting experiment on Phi-2 ({num_total_layers} layers total) on device: {device}\n")

# --- 3. Iteratively Test Layer Counts (N=1 to 32) ---
for N in range(1, num_total_layers + 1):
    start_time = time.time()

    # CRUCIAL STEP: Temporarily limit the number of active layers
    # This is done by replacing the model's layer list with a subset.
    model.model.layers = original_layers[:N]

    # Run the optimized generation function
    with torch.no_grad(): # Disable gradient calculations for faster inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True, # Use sampling for more natural text
            pad_token_id=tokenizer.eos_token_id # Ensure correct padding/stopping
        )

    # Decode and store the result
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    end_time = time.time()

    results[N] = generated_text

    print(f"--- N={N} Layer(s) | Time: {end_time - start_time:.2f}s ---")
    print(f"{generated_text}\n{'='*50}\n")

# --- 4. Restore Model and Print Summary ---
model.model.layers = original_layers # Restore all 32 layers for future use
print("Experiment complete. Model restored to full 32 layers.")

print("\n--- Summary of Layer Effect on Output Coherence (First 100 chars) ---")
for N, text in results.items():
    print(f"N={N:02d}: {text[:100].replace('\n', ' ')}...")

Starting experiment on Phi-2 (32 layers total) on device: cuda

--- N=1 Layer(s) | Time: 1.50s ---
Describe how the concept of refraction is used in the design of eyeglasses.
    # import * (akaumenalisticomposely, although not necessarily bulk ordershenekespanedictusivenessesPAC slower than any otherworldlyasca draughtminefeldglasaerobiciums fuelworkeringsIDE barter Naked

--- N=2 Layer(s) | Time: 0.36s ---
Describe how the concept of refraction is used in the design of eyeglasses.

 surprisal Unexpectedly Beckeronderesser conrstional Dedicated devoteeshipeele legenns legacies geothermal resourcefulnesses amongerstownicative bidding farewells tiobe Hamburger MXobalague Ringingledexingen

--- N=3 Layer(s) | Time: 0.44s ---
Describe how the concept of refraction is used in the design of eyeglasses.

## Templar Leighs ornamwikiabeehousevectorsiality bridge spades StyelseroydiversityCentralizedmapton RoadsuctsivelyBU-basedoff tapering existenceseUrNettingerpole Mundsicaelemagne

--- N=4 L

In [12]:
model.eval()

prompt = "Describe how the concept of refraction is used in the design of eyeglasses."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generation settings
max_new_tokens = 50
temperature = 0.7
top_p = 0.9
results = {}

# --- 2. Store Original Layers and Define Subsets ---
original_layers = model.model.layers
num_total_layers = len(original_layers)

# Define the layer subsets:
num_first_layers = 5
num_last_layers = 5

# Select the layers:
first_layers = original_layers[:num_first_layers] # Layers 0 to 4
last_layers = original_layers[-num_last_layers:]  # Layers 27 to 31 (for 32 total layers)

# Concatenate the selected layers, skipping the middle 32 - 5 - 5 = 22 layers
combined_layers = first_layers + last_layers
num_active_layers = len(combined_layers)

print(f"Starting experiment on Phi-2 ({num_total_layers} layers total) on device: {device}")
print(f"Using a discontinuous model with {num_active_layers} layers (First {num_first_layers} + Last {num_last_layers}).\n")

# --- 3. Execute Generation with Modified Layer Stack ---
start_time = time.time()

# CRUCIAL STEP: Replace the model's layers with the combined, discontinuous set
model.model.layers = combined_layers

# Run the optimized generation function
with torch.no_grad(): # Disable gradient calculations for faster inference
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True, # Use sampling for more natural text
        pad_token_id=tokenizer.eos_token_id # Ensure correct padding/stopping
    )

# Decode and store the result
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
end_time = time.time()

results['First_5_Last_5'] = generated_text

print(f"--- Combined {num_active_layers} Layers | Time: {end_time - start_time:.2f}s ---")
print(generated_text)
print("="*50)

# --- 4. Restore Model ---
model.model.layers = original_layers # Restore all 32 layers
print("\nExperiment complete. Model restored to full 32 layers.")

Starting experiment on Phi-2 (32 layers total) on device: cuda
Using a discontinuous model with 10 layers (First 5 + Last 5).

--- Combined 10 Layers | Time: 1.60s ---
Describe how the concept of refraction is used in the design of eyeglasses. The beauty of a landscape is to have a uniform and the impact on the field of geits on a woman's shoe at the bottom of a pop-like the landscape. 

Part A of the landscape design that the statement is from a

Experiment complete. Model restored to full 32 layers.
